In [5]:
import torch
import numpy as np
from torch_geometric.data import Data
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
from torch_geometric.utils import to_dense_adj
from pygsp import graphs

In [1]:
def time_step_split_helper(new_nodes, new_edges, labels):
    """
    Split the graph and store node features, edges (represented by adjacency list),
    and labels separately by timestamp t (from 1 to 49).

    Args:
        new_nodes     A dataframe of the node features
        new_edges     A dataframe of the graph's adjacency list

    Returns:
        features_t    A list of (|N_t|, d) feature matrices by timestamp
        edge_indices  A list of (2, |E_t|) adjacency list by timestamp
        labels_t      A list of (|N_t|) labels by timestamp
    """

    features =  torch.FloatTensor(new_nodes.iloc[:, 2:].to_numpy())
    times = new_nodes.iloc[:, 1].to_numpy()
    times = torch.LongTensor(times.reshape(len(times),))
    labels = labels.iloc[:, 1].to_numpy().astype(int)
    labels = torch.LongTensor(labels.reshape(len(labels),))
    labels -= 1

    nodes_id = new_nodes.iloc[:, 0].to_numpy()
    nodes_id = torch.LongTensor(nodes_id.reshape(len(nodes_id),))

    min_t = torch.min(times) # 1
    max_t = torch.max(times) # 49

    # Construct nodes of the directed graph for each time step;
    # features by time step are stored in "features_t"; labels by
    # time step are stored in "labels_t"
    features_t = []
    labels_t = []

    # Create a dictionary where
    # <key, value> = <node_id, <<idx, node_index_in_time_t_subgraph>, <t, time_t>>>.
    id2idx = {}
    for t in range(min_t, max_t + 1):
        features_t.append(features[times == t, :])
        labels_t.append(labels[times == t])
        nodes_t = nodes_id[times == t]
        for i in range(nodes_t.shape[0]):
            id2idx[nodes_t[i].item()] = {}
            id2idx[nodes_t[i].item()]['idx'] = i
            id2idx[nodes_t[i].item()]['t'] = t

    # Construct adjacency lists of the directed graph (non-symmetric) for each time step;
    # adjacency lists for each time step are stored in "edge_indices".
    edge_idx_t = [[] for _ in range(min_t, max_t + 1)]
    for index in range(new_edges.shape[0]):
        node1_t = id2idx[new_edges.iloc[index, 0]]['t']
        node1_idx = id2idx[new_edges.iloc[index, 0]]['idx']
        node2_t = id2idx[new_edges.iloc[index, 1]]['t']
        node2_idx = id2idx[new_edges.iloc[index, 1]]['idx']
        edge_idx_t[node1_t - 1].append([node1_idx, node2_idx]) # time_step starts from 1

    edge_indices = [torch.LongTensor(edge_idx_t[i]).t() for i in range(len(edge_idx_t))]
    return features_t, edge_indices, labels_t

In [7]:
def time_step_split(new_nodes, new_edges, labels, device, train_lt = 31, val_lt = 36, test_lt = 49):
    """
    Create and return the training, validation, and test set, splitted by time step,
    where each subgraph at time t is considered as an input of GCN model.

    Args:
        new_nodes     A dataframe of the node features
        new_edges     A dataframe of the graph's adjacency list
        device        Computing device
        train_lt      The last time step index of training set
        val_lt        The last time step index of validation set
        test_lt       The last time step index of test set

    Returns:
        data          A dictionary that stores training, validation, and test set,
                        each value is a list of Data object
        graph_info    A matrix where each row contains information of the time-step subgraph
                      [time_step, num_of_nodes, num_of_edges, num_of_illicit_nodes]
    """
    features_t, edge_indices, labels_t = time_step_split_helper(new_nodes, new_edges, labels)

    graph_info = np.zeros((len(labels_t), 4), dtype = np.int64)
    # for t in range(len(labels_t)):
    #     if(edge_indices[t].shape != 0):
    #         break
    #     graph_info[t, :] = np.array([t, features_t[t].shape[0], edge_indices[t].shape[1],
    #                                  labels_t[t][labels_t[t] == 1].shape[0]])

    train_idx, val_idx, test_idx = [np.arange(train_lt), np.arange(train_lt, val_lt),
                                    np.arange(val_lt, test_lt)]
    train_list = [Data(x = features_t[idx], edge_index = edge_indices[idx],
                       y = labels_t[idx]).to(device) for idx in train_idx ]
    val_list = [Data(x = features_t[idx], edge_index = edge_indices[idx],
                     y = labels_t[idx]).to(device) for idx in val_idx ]
    test_list = [Data(x = features_t[idx], edge_index = edge_indices[idx],
                      y = labels_t[idx]).to(device) for idx in test_idx ]
    data = {}
    data['train'] = train_list
    data['val'] = val_list
    data['test'] = test_list

    return data, graph_info

In [ ]:
def create_graph_helper(new_nodes, new_edges, labels):
    """
    Split the graph and store node features, edges (represented by adjacency list),
    and labels separately by timestamp t (from 1 to 49).

    Args:
        new_nodes     A dataframe of the node features
        new_edges     A dataframe of the graph's adjacency list

    Returns:
        features_t    A list of (|N_t|, d) feature matrices by timestamp
        edge_indices  A list of (2, |E_t|) adjacency list by timestamp
        labels_t      A list of (|N_t|) labels by timestamp
    """

    features =  torch.FloatTensor(new_nodes.iloc[:, 2:].to_numpy())
    times = new_nodes.iloc[:, 1].to_numpy()
    times = torch.LongTensor(times.reshape(len(times),))
    labels = labels.iloc[:, 1].to_numpy().astype(int)
    labels = torch.LongTensor(labels.reshape(len(labels),))
    labels -= 1

    nodes_id = new_nodes.iloc[:, 0].to_numpy()
    nodes_id = torch.LongTensor(nodes_id.reshape(len(nodes_id),))

    # Construct nodes of the directed graph for each time step;
    # features by time step are stored in "features_t"; labels by
    # time step are stored in "labels_t"
    features_t = features
    labels_t = labels

    # Create a dictionary where
    # <key, value> = <node_id, <<idx, node_index_in_time_t_subgraph>, <t, time_t>>>.
    id2idx = {}
    nodes_t = nodes_id

    for i in range(nodes_t.shape[0]):
        id2idx[nodes_t[i].item()] = {}
        id2idx[nodes_t[i].item()]['idx'] = i

    # Construct adjacency lists of the directed graph (non-symmetric) for each time step;
    # adjacency lists for each time step are stored in "edge_indices".
    edge_idx_t = []
    for index in range(new_edges.shape[0]):
        node1_idx = id2idx[new_edges.iloc[index, 0]]['idx']
        node2_idx = id2idx[new_edges.iloc[index, 1]]['idx']
        edge_idx_t.append([node1_idx, node2_idx]) # time_step starts from 1

    edge_indices = torch.LongTensor(edge_idx_t).t()

    return features_t, edge_indices, labels_t

In [2]:
def create_graph(nodes, edges, labels, device='cpu'):
    """
    Create and return the big graph,
    Args:
        nodes     A dataframe of the node features
        edges     A dataframe of the graph's adjacency list
        device        Computing device

    Returns:
        data          A dictionary that stores training, validation, and test set,
                        each value is a list of Data object
        graph_info    A matrix where each row contains information of the time-step subgraph
                      [time_step, num_of_nodes, num_of_edges, num_of_illicit_nodes]
    """
    features_t, edge_indices, labels_t = create_graph_helper(nodes, edges, labels)
    graph_info = np.zeros((len(labels_t), 4), dtype = np.int64)
    data = Data(x = features_t, edge_index = edge_indices, y = labels_t).to(device)

    return data, graph_info

In [8]:
@torch.no_grad()
def test(model, data, device, save_model_results=False):
    """
    Test the model by using the given split datasets.

    Args:
        model                 The GCN model
        data                  A dictionary of Data objects that store x, edge_index, and labels
                                for three sets
        save_model_results    A boolean determining whether we save the model results

    Returns
        The accuracy and auc-roc score of training, validation, and test set
    """

    model.eval()
    # The output of model on each data sets
    eval = {}
    for name in data.keys():
        data_list = data[name]
        eval_report = []
        eval_auc_roc = 0
        for i, batch in enumerate(data_list):
            out = model.forward(batch)
            y_true = batch.y
            indexes = y_true != 2

            y_pred = out.argmax(dim=-1, keepdim=True)[indexes].view(-1, 1).cpu().detach().numpy()
            y_true = y_true[indexes].view(-1, 1).cpu().detach().numpy()

            acc = classification_report(y_true, y_pred,output_dict=True, zero_division=0)
            eval_report.append(acc)
            auc_roc = roc_auc_score(y_true, y_pred)
            eval_auc_roc += auc_roc
        report = {}
        for key in eval_report[0].keys():
            if type(eval_report[0][key]) is dict:
                df = pd.DataFrame([sub_report[key] for sub_report in eval_report])
                report[key] = df.mean().to_dict()
            else:
                report[key] = np.mean(np.array([sub_report[key] for sub_report in eval_report]))
        eval_auc_roc /= len(data_list)
        eval[name] = {'report': pd.DataFrame(report), 'auc_roc': eval_auc_roc}

    ### TODO: what is the criterion to save the model results, the whole prediction
    ### y_pred and y_true? or only the test sets' prediction?
    if save_model_results:
        print ("Saving Model Predictions")

        data_new = {}
        data_new ['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

        df = pd.DataFrame(data=data_new )
        # Save locally as csv
        df.to_csv('gcn_ind.csv', sep=',', index=False)

    return eval['train']['report'], eval['val']['report'], eval['test']['report'], \
        eval['train']['auc_roc'], eval['val']['auc_roc'], eval['test']['auc_roc']

In [2]:
def train(model, train_data, optimizer, loss_fn, device):
    """
    Train the model by using the given optimizer and loss_fn.

    Args:
        model       The GCN model
        train_data  The Data object that stores x, edge_index, and labels
                      only for training set
        optimizer   The optimizer
        loss_fn     The loss function

    Returns
        The average prediction loss of each time step in the training set
          by the given loss function
    """
    model.train()
    loss = torch.FloatTensor([0]*len(train_data)).to(device)
    optimizer.zero_grad()
    for i, batch in enumerate(train_data):
        batch = batch.to(device)
        indexes = batch.y != 2

        y_pred = model.forward(batch)[indexes, :]
        y_true = batch.y[indexes]
        loss[i] = loss_fn(y_pred, y_true)
    loss.mean().backward()
    optimizer.step()
    return loss.mean().item()

In [1]:
def isConnected(edge_index):
    """
    Computes whether a graph is connected or not, based on its edge index
    :return:
    """
    # example of how to thing was implemented before and it worked
    # adj = to_dense_adj(data['train'][0].edge_index).squeeze(0)
    adj = to_dense_adj(edge_index).squeeze(0)
    g = graphs.Graph(adj)
    return g.is_connected()